# Классификация изображений природы по времени года

**Проектная работа Ситникова Андрея на курсе «Machine Learning. Professional»**

## Часть 1. Сбор данных

Большое количество бесплатных размеченных картинок можно скачать на pixabay.com.
Для использования API сайта есть даже библиотека, которой и воспользуемся.

In [1]:
#pip install python-pixabay

In [2]:
from pixabay import Image
API_KEY = '5399943-3d7bba2d8ff7c9b5846fb0950'
image = Image(API_KEY)

По одному запросу API сайта возвращает максимум 500 картинок (как выяснилось на практике, 600).

На одной странице результатов максимум 200 картинок.

Загрузим в цикле страницы по 100 картинок с тегами *summer*, *autumn*, *winter*, *spring*. Результаты объединим в список *images*.

In [6]:
from tqdm import tqdm

images = []

seasons = ['summer', 'autumn', 'winter', 'spring']
for season in seasons:
    for i in tqdm(range(1,8)):
        try:
            res = image.search(
                    q=season,
                    image_type='photo',
                    category='nature',
                    page=i,
                    per_page=100)
            for img in res['hits']:
                if img['webformatURL'].find('.png') != -1:
                    continue
                images.append({
                    'id': img['id'], 
                    'url': img['webformatURL'],
                    'season': season
                })
        except:
            print(f"Ошибка загрузки страницы {i}")

len(images)

  0%|          | 0/7 [00:00<?, ?it/s]

Ошибка загрузки страницы 7


  0%|          | 0/7 [00:00<?, ?it/s]

Ошибка загрузки страницы 7


  0%|          | 0/7 [00:00<?, ?it/s]

Ошибка загрузки страницы 7


100%|██████████| 7/7 [00:05<00:00,  1.27it/s]

Ошибка загрузки страницы 7


2388

Используем класс для многопоточного скачивания файлов.

In [7]:
import os, sys, threading
import requests

class FileDownloader():
    def __init__(self, max_threads=10):
        self.sema = threading.Semaphore(value=max_threads)
        self.headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'}
        self.block_size = 1024

    def t_getfile(self, link, filename, session):
        """ 
        Threaded function that uses a semaphore 
        to not instantiate too many threads 
        """

        self.sema.acquire()	

        filepath = os.path.join(os.getcwd() + '/Downloads/' + str(filename))
        os.makedirs(os.path.dirname(filepath), exist_ok=True)

        if not os.path.isfile(filepath):
            self.download_new_file(link, filepath, session)
        else:
            current_bytes = os.stat(filepath).st_size

            headers = requests.head(link).headers
            if 'content-length' not in headers:
                print(f"server doesn't support content-length for {link}")
                self.sema.release()
                return

            total_bytes = int(requests.head(link).headers['content-length'])

            if current_bytes < total_bytes:
                self.continue_file_download(link, filepath, session, current_bytes, total_bytes)
            else:
                print(f"already done: {filename}")

        self.sema.release()

    def download_new_file(self, link, filepath, session):
        print(f"downloading: {filepath}")
        if session == None:
            try:
                request = requests.get(link, headers=self.headers, timeout=30, stream=True)
                self.write_file(request, filepath, 'wb')
            except requests.exceptions.RequestException as e:
                print(e)
        else:
            request = session.get(link, stream=True)
            self.write_file(request ,filepath, 'wb')

    def continue_file_download(self, link, filepath, current_bytes, total_bytes):
        print(f"resuming: {filepath}")
        range_header = self.headers.copy()
        range_header['Range'] = f"bytes={current_bytes}-{total_bytes}"

        try:
            request = requests.get(link, headers=range_header, timeout=30, stream=True)
            self.write_file(request, filepath, 'ab')
        except requests.exceptions.RequestException as e:
            print(e)

    def write_file(self, content, filepath, writemode):
        with open(filepath, writemode) as f:
            for chunk in content.iter_content(chunk_size=self.block_size):
                if chunk:
                    f.write(chunk)

        print(f"completed file {filepath}", end='\n')
        f.close()

    def get_file(self, link, filename, session=None):
        """ Downloads the file"""
        thread = threading.Thread(target=self.t_getfile, args=(link, filename, session))
        thread.start()

In [8]:
downloader = FileDownloader()

for img in images:
    try:
        downloader.get_file(img['url'], f"img/{img['season']}/{img['id']}.jpg")
    except:
        print(f"Ошибка загрузки файла {img['url']}")

downloading: C:\Users\andsi\Downloads\otus\final/Downloads/img/summer/276014.jpg
downloading: C:\Users\andsi\Downloads\otus\final/Downloads/img/summer/3140492.jpg
downloading: C:\Users\andsi\Downloads\otus\final/Downloads/img/summer/3292932.jpg
downloading: C:\Users\andsi\Downloads\otus\final/Downloads/img/summer/3625405.jpg
downloading: C:\Users\andsi\Downloads\otus\final/Downloads/img/summer/3325080.jpg
downloading: C:\Users\andsi\Downloads\otus\final/Downloads/img/summer/1509956.jpg
downloading: C:\Users\andsi\Downloads\otus\final/Downloads/img/summer/2534484.jpg
downloading: C:\Users\andsi\Downloads\otus\final/Downloads/img/summer/1014712.jpg
downloading: C:\Users\andsi\Downloads\otus\final/Downloads/img/summer/2516582.jpg
downloading: C:\Users\andsi\Downloads\otus\final/Downloads/img/summer/2391348.jpg
completed file C:\Users\andsi\Downloads\otus\final/Downloads/img/summer/3625405.jpg
downloading: C:\Users\andsi\Downloads\otus\final/Downloads/img/summer/3822149.jpg
completed file 

В результате получили около 2400 картинок (около 600 по каждому времени года).

Достаточно ли этого количества, будет видно при обучении модели. Если качество обучения не удовлетворит, можно будет скачать больше картинок.

Скриншоты папок по временам года.

**Лето**:

![Image](img/summer-folder.png)

**Осень**:

![Image](img/autumn-folder.png)

**Зима**:

![Image](img/winter-folder.png)

**Весна**:

![Image](img/spring-folder.png)